# Beispiel 3.3: Reaktionsfortschritt bei Gleichgewichtsreaktionen
Bearbeitet von Franz Braun

Dieses Beispiel befindet sich im Lehrbuch auf den Seiten 17 - 19. Die Nummerierung
der verwendeten Gleichungen entspricht der Nummerierung im Lehrbuch. Das hier angewendete
Vorgehen entspricht dem im Lehrbuch vorgestellten Lösungsweg.

Zunächst werden die benötigten Pakete importiert.

In [1]:
### Import
import numpy as np
from scipy.optimize import root
from tabulate import tabulate

In diesem Beispiel wird die Gleichgewichtszusammensetzung der Ammoniaksynthese bei einer Temperatur $T$ und einem Druck $p$ betrachtet.

\begin{align*}
\mathrm{3 \, H_2 + N_2}  \leftrightarrow \mathrm{2 \, NH_3} 
\end{align*}

Zu Beginn werden  $T$, $p$ und die Stoffmengen $n_i$ zum Zeitpunkt $t = 0$ von $\mathrm{N_2}$, $\mathrm{H_2}$ und $\mathrm{NH_3}$ parametriert.

In [2]:
T       = 450           # K
p       = 4.23          # bar
n_N2_0  = 1             # mol
n_H2_0  = 1             # mol
n_NH3_0 = 1             # mol

Für die Berechnung der Gleichgewichtszusammensetzung der Ammoniaksynthese wird das Massenwirkungsgesetz aus Abschnitt 4.2 verwendet.

\begin{align*}
K_p &= \frac{p^2_\mathrm{NH_3}}{p_\mathrm{N_2} \, p^3_\mathrm{H_2}}\\
K_p \, p^2 &= \frac{x^2_\mathrm{NH_3}}{x_\mathrm{N_2} \, x^3_\mathrm{H_2}}
\end{align*}

Die Werte der Gleichgewichtskonstanten bei den gewählten Reaktionsbedingungen sind gegeben als:
\begin{align*}
K_p &= 1.397 \, \mathrm{bar^{-2}},\\
K_p \, p^2 &= 25.
\end{align*}

Laut der Definition der Reaktionslaufzahl $\xi$ und des Stoffmengenanteils $x_i$ gilt:

\begin{align*}
n_i &= n_{i,\,0} + \nu_i \xi,\\
x_i &= \frac{n_i}{\sum n_i}.
\end{align*}


Das Anwenden dieser Definitionen auf das vorliegende Reaktionsnetzwerk ergibt folgende Zusammenhänge für die Stoffmengenanteile (vgl. Tabelle 3.3 im Beispiel 3.3):

\begin{align*}
x_\mathrm{N_2} &= \frac{1 \, \mathrm{mol} - \xi} {3 \, \mathrm{mol} - 2 \, \xi},\\
x_\mathrm{H_2} &= \frac{2 \, \mathrm{mol} - 3 \, \xi} {3 \, \mathrm{mol} - 2 \, \xi},\\
x_\mathrm{NH_3} &= \frac{2 \, \xi} {3 \, \mathrm{mol} - 2 \, \xi}.
\end{align*}

Einsetzen in das Massenwirkungsgesetz und Umformen führt zu einem Nullstellenproblem:

\begin{align*}
K_p \, p^2 &= \frac{ \left(\frac{2 \, \xi} {3 \, \mathrm{mol} - 2 \, \xi}\right)^2 }{  \frac{1 \, \mathrm{mol} - \xi} {3 \, \mathrm{mol} - 2 \, \xi} \, \left( \frac{2 \, \mathrm{mol} - 3 \, \xi} {3 \, \mathrm{mol} - 2 \, \xi} \right)^3},\\
f(\xi) &= 25 \, (1 \, \mathrm{mol} - \xi) \, (2 \, \mathrm{mol} - 3\, \xi)^3 - (3 \, \mathrm{mol} - 2 \, \xi)^2 \, (2\,\xi)^2  \overset{!}{=} 0.
\end{align*}

## Lösung mit _scipy.optimize.root_

Um die Nullstellen zu bestimmten, wird $f(\xi)$ als Funktion definiert und anschließend mit _scipy.optimize.root_ nach den Nullstellen gesucht.

In [3]:
def f_xi(xi):
    '''
    Umgeformtes Massenwirkungsgesetz
     -> Funktion zur Bestimmung der Nullstellen 
    
    Parameter:
    ----------
    xi : float
        Reaktionslaufzahl in mol
    '''

    return 25 * (1 - xi) * (2 - 3 * xi)**3 - (3 - 2 * xi)**2 * (2 * xi)**2

In [4]:
xi_guess = 0                             # "initial guess" für xi
sol = root(f_xi, xi_guess, tol = 1e-10)  # Lösen mit scypi.optimize.root

''' 
Wenn der solver erfolgreich ist, wird xi ausgegeben.
Sollte das Lösen fehlschlagen, wird die "solver-message" und der "solver-success" ausgegeben.
'''
if sol.success:
    xi = sol.x[0]
    print('Das Lösen war erfolgreich.')
    print('Für die Reaktionslaufzahl wurde der Wert ', np.round(xi,3),' mol ermittelt.')
else:
    print(sol.success)
    print(sol.message)



Das Lösen war erfolgreich.
Für die Reaktionslaufzahl wurde der Wert  0.453  mol ermittelt.


Hinweis: 

Bei mehreren möglichen Lösungen hängt das Ergebnis von _scypi.optimize.root_ vom "initial-guess" ab.

## Alternative Lösung mit dem NEWTONschen Verfahren

Alternativ zu _scipy.optimize.root_ kann das NEWTONsche Verfahren, wie im Lehrbuch beschrieben, verwendet werden.

Hierfür wird die Ableitung $f'(\xi)$ benötigt:

\begin{align*}
f'(\xi) &= - 25 \, \left[ (2 \, \mathrm{mol} - 3 \, \xi)^3 + 9 \, (1 \, \mathrm{mol} - \xi) \, (2 \, \mathrm{mol} - 3 \, \xi)^2 \right] 
+ 4 \, (3 \, \mathrm{mol} - 2 \, \xi) \, (2\,\xi)^2 - (3 \, \mathrm{mol} - 2\, \xi)^2 \, 8\,\xi.
\end{align*}

In [5]:
def df_xi(xi):
    '''
    Ableitung des umgeformten Massenwirkungsgesetzes
     -> Funktion zur Bestimmung der Nullstellen 
    
    Parameter:
    ----------
    xi : float
        Reaktionslaufzahl in mol
    '''

    # return - 25 * ((2 - 3 * xi)**3 + 9 * (1 - xi) * (2 - 3 * xi)**2) + 4 * (3 - 2 * xi) * (2 * xi)**2 - (3 - 2 * xi)**2 * 8 * xi
    return 2636 * xi**3 - 5931 * xi**2 + 4428 * xi - 1100

Das Verfahren wird mit folgender Iterationsvorschrift durchgeführt. Dabei können $n$ Iterationsschritte verwendet werden, bis der Unterschied zwischen zwei aufeinanderfolgenden Schritten, $\Delta x = x_{n+1} - x_n$, kleiner als ein zuvor festgelegter Wert, der Threshold, ist. 
\begin{align*}
x_{n+1} &= x_n - \frac{f(x_n)}{f'(x_n)}
\end{align*}

Da es sich um ein iteratives Verfahren handelt, erfolgt das Lösen mit einer _while_ Bedingung. Der erste Iterationsschritt wird vor der Bedingung durchgeführt und liefert das erste $\Delta x$ zum Vergleich mit dem Threshold. 

Hinweis : Es sollte neben dem Threshold ein weiteres Abbruchkriterium der Bedingung definiert werden, um eine (unendlich) lange Rechenzeit zu vermeiden. Häufig wird hierfür eine maximale Anzahl an Iterationsschritten vorgegeben. 

In [6]:
threshold       = 1e-10 # Threshold
i               = 0     # Variable zum Zählen der Iterationsschritte
x_0             = 0.3   # Startwert
max_it          = 1e5   # Maximale Anzahl an Iterationsschritten

### Iteration
x = [] # Liste zum Speichern der ermittelten xi

# erster Iterationsschritt
x.append(x_0)
x.append(x[-1] - f_xi(x[-1] ) / df_xi(x[-1]))
delta_x         = x[-1] - x[-2]
i               += 1

# alle weiteren Iterationsschritte
while (delta_x > threshold and i < max_it):
   
    x.append (x[-1] - f_xi(x[-1] ) / df_xi(x[-1]))
    delta_x     = x[-1] - x[-2]
    i           += 1  
    if i == 501:
        print(x)
        raise ValueError('Maximale Anzahl an Iterationen erreicht.')

# Der letzte Wert in der Liste ist die Lösung
xi_newton = x[-1]


print('Für die Reaktionslaufzahl wurde der Wert ', np.round(xi_newton, 3),' mol mit', i ,' Iterationen ermittelt.')
print('Der relative Unterschied der beiden Lösungsverfahren beträgt: ', (xi-xi_newton)/xi_newton)

Für die Reaktionslaufzahl wurde der Wert  0.453  mol mit 7  Iterationen ermittelt.
Der relative Unterschied der beiden Lösungsverfahren beträgt:  0.0


Es werden die gleichen Ergebnisse mit beiden Lösungsverfahren erzielt. 

## Berechnung der Stoffmengen und Stoffmengenanteile im Gleichgewicht

Abschließend werden die Stoffmengen und Stoffmengenanteile im Gleichgewicht bestimmt.

\begin{align*}
n_\mathrm{N2,\,eq}     &= 1\, \mathrm{mol} - \xi   \\  
n_\mathrm{H2,\,eq}     &= 2\, \mathrm{mol} - 3 \, \xi\\
n_\mathrm{NH_3,\,eq}   &= 2 \, \xi\\
x_{i,\,\mathrm{eq}} &= \frac{n_{i,\,\mathrm{eq}}}{n_\mathrm{N2,\,eq} + n_\mathrm{H2,\,eq} + n_\mathrm{NH_3,\,eq}}
\end{align*}

In [7]:
# Stoffmengen in mol
n_N2_eq     = 1 - xi     
n_H2_eq     = 2 - 3 * xi
n_NH_3_eq   = 2 * xi

# Stoffmengenanteile
x_N2_eq     = n_N2_eq   / (n_N2_eq + n_H2_eq + n_NH_3_eq)
x_H2_eq     = n_H2_eq   / (n_N2_eq + n_H2_eq + n_NH_3_eq)
x_NH_3_eq   = n_NH_3_eq / (n_N2_eq + n_H2_eq + n_NH_3_eq)

Die Ausgabe der Ergebnisse erfolgt tabellarisch.

In [8]:
# Ausgabe der Ergebnisse als Tabelle

names_y     = np.array((['n_i / mol'], ['x_i / 1']))
n_row       = np.array((n_N2_eq, n_H2_eq, n_NH_3_eq))  
x_row       = np.array((x_N2_eq, x_H2_eq, x_NH_3_eq))

# Array für die Tabelle
table       = np.hstack((names_y, np.round(np.vstack((n_row, x_row)), 3)))

print(tabulate(table, headers = ['N_2', 'H_2', 'NH_3'] ))

             N_2    H_2    NH_3
---------  -----  -----  ------
n_i / mol  0.547  0.641   0.906
x_i / 1    0.261  0.306   0.433
